In [1]:
import torch
import torch.nn as nn

seed = 42
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### NLP Example

* Sample Embedding 생성

In [2]:
batch, sentence_len, embed_dim = 2, 4, 5
batch_embed = torch.randn(batch, embed_dim, sentence_len)
print(batch_embed)

tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055],
         [ 0.6784, -1.2345, -0.0431, -1.6047],
         [-0.7521,  1.6487, -0.3925, -1.4036],
         [-0.7279, -0.5594, -0.7688,  0.7624],
         [ 1.6423, -0.1596, -0.4974,  0.4396]],

        [[-0.7581,  1.0783,  0.8008,  1.6806],
         [ 0.0349,  0.3211,  1.5736, -0.8455],
         [ 1.3123,  0.6872, -1.0892, -0.3553],
         [-1.4181,  0.8963,  0.0499,  2.2667],
         [ 1.1790, -0.4345, -1.3864, -1.2862]]])


* nn.BatchNorm1d 함수를 사용한 Nomalize 
    * batch을 기준으로 $\mu, \sigma^2$를 이용하여 Normalize
    * nn.BatchNorm1d은 입력을 (batch, embed_dim, sentence_len)으로 받으므로 transpose를 통해 (batch, sentence_len, embed_dim)으로 형태를 변경
    * batch 내의 동일한 위치의 embedding 값을 기준으로 $\mu, \sigma^2$를 계산
        * eg. $\mu_{1} = \frac{(embed_{1,1} + embed_{2,1} + embed_{3,1} + ... + embed_{batch size, 1})}{batch size}$


In [ ]:
batch_1d_norm = nn.BatchNorm1d(embed_dim)
output = batch_1d_norm(batch_embed)
print(output)

tensor([[[ 1.0112,  0.6694,  0.2133, -2.1240],
         [ 0.8342, -1.1157,  0.0988, -1.4930],
         [-0.6723,  1.6039, -0.3313, -1.2899],
         [-0.7092, -0.5581, -0.7460,  0.6279],
         [ 1.6754, -0.0950, -0.4269,  0.4937]],

        [[-1.0764,  0.3514,  0.1356,  0.8197],
         [ 0.1783,  0.4700,  1.7466, -0.7191],
         [ 1.2850,  0.6923, -0.9918, -0.2960],
         [-1.3285,  0.7479, -0.0114,  1.9775],
         [ 1.2202, -0.3651, -1.3004, -1.2020]]],
       grad_fn=<NativeBatchNormBackward0>)


* Batch Normalization 구현 
* $y = \frac{x - E[x]}{\sqrt{Var[x] + \epsilon}}*\gamma + \beta$
* $\gamma, \beta$는 학습 가능한 paramters( weight, bias )
* var 계산 시, unbiased=False으로 설정하지 않으면 Bessel’s correction을 통해 표본 크기를 n이 아닌 n-1을 사용하게 된다)  

In [ ]:
batch_embed = batch_embed.transpose(1, 2)
eg_mean = torch.mean(batch_embed, dim=(0, 1))
print('mean:\n ', eg_mean)
print(' ')
eg_var = torch.var(batch_embed, dim=(0, 1), unbiased = False)
print('var:\n', eg_var)

mean:
  tensor([ 0.6264, -0.1400, -0.0431,  0.0626, -0.0629])
 
var:
 tensor([1.6543, 0.9625, 1.1125, 1.2423, 1.0358])


In [ ]:
# 위의 결과와 동일
eg_x_hat = (batch_embed - eg_mean) / torch.sqrt(eg_var + batch_1d_norm.eps)
print((batch_1d_norm.weight * eg_x_hat + batch_1d_norm.bias).transpose(1, 2))

tensor([[[ 1.0112,  0.6694,  0.2133, -2.1240],
         [ 0.8342, -1.1157,  0.0988, -1.4930],
         [-0.6723,  1.6039, -0.3313, -1.2899],
         [-0.7092, -0.5581, -0.7460,  0.6279],
         [ 1.6754, -0.0950, -0.4269,  0.4937]],

        [[-1.0764,  0.3514,  0.1356,  0.8197],
         [ 0.1783,  0.4700,  1.7466, -0.7191],
         [ 1.2850,  0.6923, -0.9918, -0.2960],
         [-1.3285,  0.7479, -0.0114,  1.9775],
         [ 1.2202, -0.3651, -1.3004, -1.2020]]], grad_fn=<TransposeBackward0>)


### Image Example

In [ ]:
batch, channel, height, width = 2, 3, 5, 5
batch_img = torch.randn(batch, channel, height, width)
print(batch_img)

tensor([[[[-0.8371, -0.9224,  1.8113,  0.1606,  0.3672],
          [ 0.1754,  1.3852, -0.4459, -1.2024,  0.7078],
          [-1.0759,  0.5357,  1.1754,  0.5612, -0.4527],
          [-0.7718,  0.1453,  0.2311,  0.0087, -0.1423],
          [ 0.1971, -1.1441,  0.3383,  1.6992,  2.8140]],

         [[ 0.3598, -0.0898,  0.4584, -0.5644,  1.0563],
          [-1.4692,  1.4332,  0.7281, -0.7106, -0.6021],
          [ 0.9604,  0.4048, -1.3543, -0.4976,  0.4747],
          [-0.1976,  1.2683,  1.2243,  0.0981,  1.7423],
          [-1.3527,  0.2191,  0.5526, -0.6788,  0.5743]],

         [[ 0.1877, -0.3576, -0.3165,  0.5886, -0.8905],
          [ 0.4098, -0.9864,  0.1233,  0.3499,  0.6173],
          [-0.1693,  0.2332,  4.0356,  1.2795,  1.0311],
          [-0.7048,  1.0131, -0.3308,  0.5177,  0.3878],
          [-0.5797, -0.1691, -0.5733,  0.5069, -0.4752]]],


        [[[-0.4920,  0.2704, -0.5628,  0.6793,  0.4405],
          [-0.3609, -0.0606,  0.0733,  0.8187,  1.4805],
          [ 0.3449, -1.

* nn.BatchNorm1d 함수를 사용한 Nomalize 
    * batch을 기준으로 $\mu, \sigma^2$를 이용하여 Normalize
    * nn.BatchNorm1d은 입력을 (batch, channel, height, width)으로 받으므로 permute를 통해 (batch, height, width, channel)으로 형태를 변경
    * batch 내의 동일한 위치의 channel 값을 기준으로 $\mu, \sigma^2$를 계산
        * eg. $\mu_{1} = \frac{(channel{1,1,1} + channel{1,2,1} + channel{1,3,1} + ... + channel{1,batch size, 1})}{batch size}$


In [ ]:
batch_2d_norm = nn.BatchNorm2d(channel)
print(batch_2d_norm(batch_img))

tensor([[[[-0.9478, -1.0392,  1.8916,  0.1218,  0.3434],
          [ 0.1378,  1.4347, -0.5283, -1.3394,  0.7085],
          [-1.2037,  0.5240,  1.2098,  0.5513, -0.5357],
          [-0.8777,  0.1055,  0.1974, -0.0410, -0.2028],
          [ 0.1610, -1.2769,  0.3124,  1.7714,  2.9666]],

         [[ 0.4075, -0.1167,  0.5225, -0.6700,  1.2195],
          [-1.7248,  1.6589,  0.8369, -0.8405, -0.7139],
          [ 1.1078,  0.4600, -1.5909, -0.5921,  0.5414],
          [-0.2423,  1.4667,  1.4154,  0.1024,  2.0193],
          [-1.5890,  0.2434,  0.6323, -0.8033,  0.6576]],

         [[ 0.1779, -0.3951, -0.3519,  0.5991, -0.9549],
          [ 0.4113, -1.0557,  0.1102,  0.3483,  0.6292],
          [-0.1972,  0.2257,  4.2208,  1.3250,  1.0640],
          [-0.7598,  1.0452, -0.3669,  0.5246,  0.3881],
          [-0.6284, -0.1970, -0.6217,  0.5132, -0.5186]]],


        [[[-0.5778,  0.2396, -0.6537,  0.6780,  0.4220],
          [-0.4372, -0.1153,  0.0282,  0.8274,  1.5369],
          [ 0.3195, -1.

In [ ]:
batch_img = batch_img.permute(0, 2, 3, 1)
eg_mean = torch.mean(batch_img, dim=(0, 1, 2), keepdim=True)  # E[x]
print('mean:\n ', eg_mean)
print(' ')
eg_var = torch.var(batch_img, dim=(0, 1, 2), keepdim=True, unbiased=False) # Var[x]
print('var:\n', eg_var)

mean:
  tensor([[[[0.0469, 0.0103, 0.0184]]]])
 
var:
 tensor([[[[0.8700, 0.7357, 0.9058]]]])


In [ ]:
# 위의 결과와 동일
eg_x_hat = (batch_img - eg_mean) / torch.sqrt(eg_var + batch_2d_norm.eps)
print((batch_2d_norm.weight * eg_x_hat + batch_2d_norm.bias).permute(0, 2, 3, 1))

tensor([[[[-0.9478,  0.1378, -1.2037, -0.8777,  0.1610],
          [ 0.4075, -1.7248,  1.1078, -0.2423, -1.5890],
          [ 0.1779,  0.4113, -0.1972, -0.7598, -0.6284]],

         [[-1.0392,  1.4347,  0.5240,  0.1055, -1.2769],
          [-0.1167,  1.6589,  0.4600,  1.4667,  0.2434],
          [-0.3951, -1.0557,  0.2257,  1.0452, -0.1970]],

         [[ 1.8916, -0.5283,  1.2098,  0.1974,  0.3124],
          [ 0.5225,  0.8369, -1.5909,  1.4154,  0.6323],
          [-0.3519,  0.1102,  4.2208, -0.3669, -0.6217]],

         [[ 0.1218, -1.3394,  0.5513, -0.0410,  1.7714],
          [-0.6700, -0.8405, -0.5921,  0.1024, -0.8033],
          [ 0.5991,  0.3483,  1.3250,  0.5246,  0.5132]],

         [[ 0.3434,  0.7085, -0.5357, -0.2028,  2.9666],
          [ 1.2195, -0.7139,  0.5414,  2.0193,  0.6576],
          [-0.9549,  0.6292,  1.0640,  0.3881, -0.5186]]],


        [[[-0.5778, -0.4372,  0.3195, -1.5872,  0.5087],
          [-0.5083, -0.3969,  1.6915,  1.2914,  1.1320],
          [-1.0318,

### Momentum

In [ ]:
batch_2d_norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1.])),
             ('bias', tensor([0., 0., 0.])),
             ('running_mean', tensor([0.0047, 0.0010, 0.0018])),
             ('running_var', tensor([0.9888, 0.9751, 0.9924])),
             ('num_batches_tracked', tensor(1))])

* batchnorm에는 layernorm과 달리 momentum 값과 running_mean, running_var 값이 존재한다.
    * 이는 이동평균을 계산하기 위함으로써 학습때는 사용되지 않고 갱신만 되다가 model.training = False인 validation이나 test에서 사용된다.

* 최초 running_mean 값은 torch.zeros(dim), 최초 running_var 값은 torch.ones(dim)이며 아래 코드와 같이 업데이트 된다
    * with torch.no_grad():
        * self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * eg_mean(batch_mean)
        * self.running_var = (1 - self.momentum) * self.running_var + self.momentum * eg_var(batch_var)

* momentum 값이 작을수록 이전 batch의 평균과 분산이 현재 batch의 평균과 분산에 더 많은 영향을 미치게 된다. default = 0.1